In [67]:
import pandas as pd
import time
import numpy as np
from scipy import stats
import json

In [68]:
n='去异常 未归一N-all.csv'
p='去异常 未归一P-all.csv'

In [69]:

def read(tit):
    path = 'E:\\desktop\\data\\app\\223\\fea\\'

    #读取数据
    try:
        data = pd.read_csv(path+tit,encoding='gbk',index_col=0)
    except:
        data = pd.read_csv(path+tit,encoding='utf-8',index_col=0)

    return data

n_df=read(n)
p_df=read(p)

fea_df=pd.concat([n_df,p_df],axis=0)


In [70]:
fea_list=list(fea_df.columns)
norm_dict={}
for fea in fea_list:
    col=list(fea_df[fea])
    higher = np.quantile(col, 0.75, interpolation='higher')  # 上四分位
    lower = np.quantile(col, 0.25, interpolation='lower')  # 下四分位
    iqr = higher - lower  # 四分位距
    low, up = lower - 2 * iqr, higher + 2 * iqr
    low_,up_= lower - 1 * iqr, higher + 1 * iqr
    
    for i in range(len(col)):
        if col[i] > up:
            col[i] = up_
        elif col[i] < low:
            col[i] = low_
            
    norm_dict[fea]=(min(col),max(col),lower,higher)

# norm_dict

#### 保存normdict

In [71]:

def default_dump(obj):
    """Convert numpy classes to JSON serializable objects."""
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj
norm_json=json.dumps(norm_dict,default=default_dump)
with open('norm.json','w+') as file:
    file.write(norm_json)

#### 保存归一化后的文件

In [72]:
def pure3(fea_dict,norm_dict=norm_dict):
    for fea in fea_dict.keys():
        iqr=norm_dict[fea][3]-norm_dict[fea][2]
        low, up = norm_dict[fea][2] - 2 * iqr, norm_dict[fea][3] + 2 * iqr
        low_,up_= norm_dict[fea][2] - 1 * iqr, norm_dict[fea][3] + 1 * iqr
        value=fea_dict[fea]
        if value > up:
            fea_dict[fea] = up_
        elif value < low:
            fea_dict[fea] = low_

    return fea_dict


# 归一化
def normalization(fea_dict,norm_dict=norm_dict):

    for fea in fea_dict.keys():
        value = fea_dict[fea]
        if value > norm_dict[fea][1]:
            fea_dict[fea] = 1
        elif value < norm_dict[fea][0]:
            fea_dict[fea] = 0
        else:
            fea_dict[fea] = (value-norm_dict[fea][0])/(norm_dict[fea][1]-norm_dict[fea][0])

    return fea_dict

In [73]:
def go(data):
    index=data.index
    fea_list=list(data.columns)
    fea_dict_all={}
    for i in range(len(data)):
        fea_dict=dict(data.iloc[i])
        # fea_dict=pure3(fea_dict)
        fea_dict=normalization(fea_dict)

        for fea in fea_list:
            if i ==0:
                fea_dict_all[fea]=[]

            fea_dict_all[fea].append(fea_dict[fea])
    fea_df=pd.DataFrame(fea_dict_all,index=index)
    return fea_df

go(n_df).to_csv("E:\\desktop\\data\\app\\fea\\N_fea_11.csv",encoding="gbk")
go(p_df).to_csv("E:\\desktop\\data\\app\\fea\\P_fea_11.csv",encoding="gbk")